In [2]:
import sys
sys.path.append("../code/")
import numpy as np
import pandas as pd
from dataloader import APPLIANCE_ORDER, get_train_test
from ddsc import SparseCoding, reshape_for_sc
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

## sc without discreminative

In [4]:
error = {}
for dataset in [1 ,2]:
    error[dataset] = {}
    for cur_fold in range(5):
        error[dataset][cur_fold] = {}
        for num_latent in range(1, 21):
            error[dataset][cur_fold][num_latent] = np.load("../code/baseline/sc-non-nested/sc-non-error-{}-{}-{}.npy".format(dataset, cur_fold, num_latent)).item()

In [13]:
best_param = {}
for dataset in [1, 2]:
    best_param[dataset] = {}
    for cur_fold  in range(5):
        best_param[dataset][cur_fold] = {}
        
        best_latent = pd.DataFrame(error[dataset][cur_fold]).mean().idxmin()
        best_param[dataset][cur_fold]['num_latent'] = best_latent

In [16]:
np.save("../code/baseline/best_param_nested_sc_non.npy", best_param)

## sc With Discreminative

In [24]:
error = {}
for dataset in [1 ,2]:
    error[dataset] = {}
    for cur_fold in range(5):
        error[dataset][cur_fold] = {}
        for num_latent in range(1, 21):
            error[dataset][cur_fold][num_latent] = {}
            for iters in range(10, 110, 10):
                error[dataset][cur_fold][num_latent][iters] = np.load("../code/baseline/sc-with-nested/sc-with-error-{}-{}-{}-{}.npy".format(dataset, cur_fold, num_latent, iters)).item()

In [25]:
pd.Series(error[dataset][cur_fold][num_latent][iters]).mean()

160.95381271488128

In [26]:
best_param = {}
for dataset in [1, 2]:
    best_param[dataset] = {}
    for cur_fold  in range(5):
        best_param[dataset][cur_fold] = {}
        min_error = np.inf
        for num_latent in range(1, 21):
            for iters in range(10, 110, 10):
                cur_error = pd.Series(error[dataset][cur_fold][num_latent][iters]).mean()
                if cur_error < min_error:
                    min_error = cur_error
                    best_param[dataset][cur_fold]['num_latent'] = num_latent
                    best_param[dataset][cur_fold]['iters'] = iters

In [28]:
np.save("../code/baseline/best_param_nested_sc_with.npy", best_param)

# prediction

In [29]:
def discriminative(dataset, cur_fold, num_latent, num_iterations):
#     out = []
    num_folds = 5
    
    train, test = get_train_test(dataset, num_folds=num_folds, fold_num=cur_fold)
    train_sc, test_sc = reshape_for_sc(train), reshape_for_sc(test)
    train_data = np.array([train_sc[:, :, i ] for i in range(1, train.shape[1])]).swapaxes(1, 2)
    c = SparseCoding()
    c.train(train_data, num_latent=num_latent)
    pred = c.disaggregate_discriminative(train_sc[:, :, 0].swapaxes(0, 1), 
                                         test_sc[:, :, 0].swapaxes(0, 1),
                                         num_iter=num_iterations)
    pred = pred[-1, :, :, :]
    pred = pred.swapaxes(0, 2).swapaxes(1, 2)
    print(pred.shape)
    pred = pred.reshape(pred.shape[0], pred.shape[1], -1, 24)
#     out.append(pred)
    
#     return np.concatenate(out)
    return pred

In [30]:

best_param = np.load("../code/baseline/best_param_nested_sc_with.npy").item()
for dataset in [1, 2]:
    out = []
    for cur_fold in range(5):
        pred = discriminative(dataset, cur_fold, best_param[dataset][cur_fold]['num_latent'], best_param[dataset][cur_fold]['iters'])
        out.append(pred)
        
    cont_out = np.concatenate(out)
    np.save("../code/baseline/sc-with-nested-pred-{}.npy".format(dataset), cont_out)
    
    if dataset == 1:
        tensor = np.load("../2015-5appliances.numpy.npy")
    if dataset == 2:
        tensor = np.load("../2015-5appliances-true-agg.npy")
    
    gt = tensor[:, 1:, :, :]
    cont_out = np.minimum(cont_out, tensor[:, 0:1, :, :])
    err_disc = {APPLIANCE_ORDER[i+1]:mean_absolute_error(cont_out[:, i,:,:].flatten(), 
                                                                       gt[:, i, :, :].flatten()) for i in range(cont_out.shape[1])}
    
    np.save("../code/baseline/sc-with-nested-error-{}.npy".format(dataset), err_disc)

(14, 5, 2688)
(14, 5, 2688)
(14, 5, 2688)
(13, 5, 2688)
(13, 5, 2688)
(14, 5, 2688)
(14, 5, 2688)
(14, 5, 2688)
(13, 5, 2688)
(13, 5, 2688)


In [33]:
err_1 = np.load("../code/baseline/sc-with-nested-error-2.npy")
err_1

array({'hvac': 635.64061321300312, 'fridge': 213.77657813058056, 'dr': 114.64303711171128, 'dw': 19.476897946856177, 'mw': 20.453114873057086}, dtype=object)

In [39]:
def non_discriminative(dataset, cur_fold, num_latent):
    num_folds=5
    train, test = get_train_test(dataset, num_folds=num_folds, fold_num=cur_fold)
    train_sc, test_sc = reshape_for_sc(train), reshape_for_sc(test)
    train_data = np.array([train_sc[:, :, i ] for i in range(1, train.shape[1])]).swapaxes(1, 2)
    c = SparseCoding()
    c.train(train_data, num_latent=num_latent)
    pred = c.disaggregate(test_sc[:, :, 0].swapaxes(0, 1)).swapaxes(0, 2).swapaxes(1, 2)
    pred = pred.reshape(pred.shape[0], pred.shape[1], -1, 24)
#     out.append(pred)
    
    return pred

In [40]:

best_param = np.load("../code/baseline/best_param_nested_sc_non.npy").item()
for dataset in [1, 2]:
    out = []
    for cur_fold in range(5):
        pred = non_discriminative(dataset, cur_fold, best_param[dataset][cur_fold]['num_latent'])
        out.append(pred)
        
    cont_out = np.concatenate(out)
    np.save("../code/baseline/sc-non-nested-pred-{}.npy".format(dataset), cont_out)
    
    if dataset == 1:
        tensor = np.load("../2015-5appliances.numpy.npy")
    if dataset == 2:
        tensor = np.load("../2015-5appliances-true-agg.npy")
    
    gt = tensor[:, 1:, :, :]
    cont_out = np.minimum(cont_out, tensor[:, 0:1, :, :])
    err_disc = {APPLIANCE_ORDER[i+1]:mean_absolute_error(cont_out[:, i,:,:].flatten(), 
                                                                       gt[:, i, :, :].flatten()) for i in range(cont_out.shape[1])}
    
    np.save("../code/baseline/sc-non-nested-error-{}.npy".format(dataset), err_disc)

In [42]:
err_1 = np.load("../code/baseline/sc-non-nested-error-1.npy")
err_1

array({'hvac': 260.31440724551442, 'fridge': 106.38220734255049, 'dr': 88.89888812445281, 'dw': 35.113502253401194, 'mw': 46.458593872658582}, dtype=object)